In [ ]:
import sqlite3
import pandas as pd
from datetime import datetime, timedelta

In [ ]:
# Создание подключения к базе данных SQLite
conn = sqlite3.connect('rides_database.db')

# Создание курсора
cursor = conn.cursor()

# Создание таблиц
cursor.executescript('''
CREATE TABLE IF NOT EXISTS Cities (
    id INTEGER PRIMARY KEY,
    title TEXT NOT NULL,
    district INTEGER,  -- FK
    active BIT
);

CREATE TABLE IF NOT EXISTS Districts (
    id INTEGER PRIMARY KEY,
    title TEXT NOT NULL,
    active BIT
);

CREATE TABLE IF NOT EXISTS Passengers (
    id INTEGER PRIMARY KEY,
    fullname TEXT NOT NULL,
    city INTEGER,  -- FK
    district INTEGER,  -- FK
    pan_card TEXT NOT NULL,
    active BIT
);

CREATE TABLE IF NOT EXISTS Rides (
    id INTEGER PRIMARY KEY,
    start_city INTEGER,  -- FK
    end_city INTEGER,    -- FK
    pan_card TEXT NOT NULL,
    start_time DATETIME,
    end_time DATETIME,
    cost REAL,
    active BIT
);
''')

# Сохранить изменения и закрыть курсор
conn.commit()

In [ ]:
# Заполнение таблиц по три записи

# Заполнение таблицы Cities
cursor.executescript('''
INSERT INTO Cities (title, district, active) VALUES
('City A', 1, 1),
('City B', 1, 1),
('City C', 1, 1);
''')

# Заполнение таблицы Districts
cursor.executescript('''
INSERT INTO Districts (title, active) VALUES
('District 1', 1),
('District 2', 1),
('District 3', 1);
''')

# Заполнение таблицы Passengers
cursor.executescript('''
INSERT INTO Passengers (fullname, city, district, pan_card, active) VALUES
('Passenger 1', 1, 1, 'PC001', 1),
('Passenger 2', 2, 1, 'PC002', 1),
('Passenger 3', 3, 1, 'PC003', 1);
''')

# Данные для таблицы Rides
cursor.executescript('''
INSERT INTO Rides (start_city, end_city, pan_card, start_time, end_time, cost, active) VALUES
(1, 2, 'PC001', '2023-01-15 10:00:00', '2023-01-15 11:00:00', 100.0, 1),
(2, 3, 'PC002', '2023-01-16 12:00:00', '2023-01-16 13:00:00', 200.0, 1),
(1, 3, 'PC001', '2023-02-17 14:00:00', '2023-02-17 15:00:00', 150.0, 1),
(1, 3, 'PC002', '2023-03-01 09:00:00', '2023-03-01 09:30:00', 120.0, 1),
(1, 2, 'PC003', '2023-03-05 14:00:00', '2023-03-05 15:00:00', 130.0, 1),
(2, 3, 'PC001', '2023-03-10 16:00:00', '2023-03-10 17:00:00', 210.0, 1);
''')

# Сохранить изменения и закрыть курсор
conn.commit()


In [ ]:
# 1. Количество и стоимость поездок по каждому городу (старты) по месяцам текущего года

query1 = '''
SELECT
    C.title AS city,
    strftime('%Y-%m', R.start_time) AS month,
    COUNT(*) AS ride_count,
    SUM(R.cost) AS total_cost
FROM
    Rides R
JOIN
    Cities C ON R.start_city = C.id
WHERE
    strftime('%Y', R.start_time) = strftime('%Y', 'now')
GROUP BY
    C.title, month;
'''

# Выполнение запроса и вывод результатов
df1 = pd.read_sql_query(query1, conn)
print("Количество и стоимость поездок по каждому городу по месяцам текущего года:")
print(df1)

Количество и стоимость поездок по каждому городу по месяцам текущего года:
Empty DataFrame
Columns: [city, month, ride_count, total_cost]
Index: []


In [ ]:
# 2. Количество и стоимость поездок по каждому городу по месяцам текущего года в сравнении с аналогичными периодами прошлого года

query2 = '''
SELECT
    C.title AS city,
    strftime('%Y-%m', R.start_time) AS month,
    COUNT(*) AS ride_count,
    SUM(R.cost) AS total_cost,
    strftime('%Y', R.start_time) AS year
FROM
    Rides R
JOIN
    Cities C ON R.start_city = C.id
WHERE
    strftime('%Y', R.start_time) IN (strftime('%Y', 'now'), strftime('%Y', 'now', '-1 year'))
GROUP BY
    C.title, month, year;
'''

# Выполнение запроса и вывод результатов
df2 = pd.read_sql_query(query2, conn)
print("\nКоличество и стоимость поездок по каждому городу по месяцам текущего года в сравнении с аналогичными периодами прошлого года:")
print(df2)


Количество и стоимость поездок по каждому городу по месяцам текущего года в сравнении с аналогичными периодами прошлого года:
     city    month  ride_count  total_cost  year
0  City A  2023-01           2       200.0  2023
1  City A  2023-02           2       300.0  2023
2  City A  2023-03           4       500.0  2023
3  City B  2023-01           2       400.0  2023
4  City B  2023-03           2       420.0  2023


In [ ]:
# 3. Пассажиры, у которых больше max_rides_per_day поездок в день за последний месяц

max_rides_per_day = 2  # Устанавливаем переменную для ограничения

query3 = f'''
SELECT
    P.fullname,
    COUNT(*) / COUNT(DISTINCT DATE(R.start_time)) AS rides_per_day
FROM
    Rides R
JOIN
    Passengers P ON R.pan_card = P.pan_card
WHERE
    R.start_time >= DATE('now', 'start of month', '-1 month')
GROUP BY
    P.fullname
HAVING
    rides_per_day > {max_rides_per_day};
'''

# Выполнение запроса и вывод результатов
df3 = pd.read_sql_query(query3, conn)
print("\nПассажиры с количеством поездок больше, чем max_rides_per_day за последний месяц:")
print(df3)


Пассажиры с количеством поездок больше, чем max_rides_per_day за последний месяц:
Empty DataFrame
Columns: [fullname, rides_per_day]
Index: []


In [ ]:
# 4. Количество стартов и завершений поездок по районам за последний месяц

query4 = '''
SELECT
    D.title AS district,
    COUNT(CASE WHEN R.start_time >= DATE('now', 'start of month', '-1 month') THEN 1 END) AS starts,
    COUNT(CASE WHEN R.end_time >= DATE('now', 'start of month', '-1 month') THEN 1 END) AS completions
FROM
    Rides R
JOIN
    Cities C ON R.start_city = C.id
JOIN
    Districts D ON C.district = D.id
WHERE
    R.start_time >= DATE('now', 'start of month', '-1 month')
    OR R.end_time >= DATE('now', 'start of month', '-1 month')
GROUP BY
    D.title;
'''

# Выполнение запроса и вывод результатов
df4 = pd.read_sql_query(query4, conn)
print("\nКоличество стартов и завершений поездок по районам за последний месяц:")
print(df4)


Количество стартов и завершений поездок по районам за последний месяц:
Empty DataFrame
Columns: [district, starts, completions]
Index: []


In [ ]:
# Рекомендуемые индексы
cursor.executescript('''
CREATE INDEX IF NOT EXISTS idx_rides_start_time ON Rides(start_time);
CREATE INDEX IF NOT EXISTS idx_rides_end_time ON Rides(end_time);
CREATE INDEX IF NOT EXISTS idx_passengers_pan_card ON Passengers(pan_card);
CREATE INDEX IF NOT EXISTS idx_cities_district ON Cities(district);
CREATE INDEX IF NOT EXISTS idx_districts_title ON Districts(title);
''')

# Сохранить изменения
conn.commit()

In [ ]:
# Закрытие подключения
conn.close()

1. idx_rides_start_time:
- Описание: Этот индекс создаётся на поле start_time таблицы Rides.
- Назначение: Ускоряет выполнение запросов, которые фильтруют или сортируют данные по времени начала поездки (например, в условиях, где необходимо выбрать поездки за определённый период времени).

2. idx_rides_end_time:
- Описание: Индекс на поле end_time таблицы Rides.
- Назначение: Подобно предыдущему индексу, он оптимизирует запросы, которые работают с временем окончания поездки, что позволяет быстрее фильтровать или сортировать данные по этому полю.

3. idx_passengers_pan_card:
- Описание: Индекс на поле pan_card в таблице Passengers.
- Назначение: Улучшает производительность запросов, которые используют номер карты пассажира для фильтрации данных, например, при присоединении таблицы Rides к Passengers.

4. idx_cities_district:
- Описание: Индекс на поле district в таблице Cities.
- Назначение: Ускоряет выборку городов по району, что полезно в запросах, где необходимо объединить информацию о городах и районах и фильтровать результаты по диапазону значений района.

5. idx_districts_title:
- Описание: Индекс на поле title в таблице Districts.
- Назначение: Повышает производительность запросов, которые фильтруют или сортируют данные по названию района. Это может быть полезно, если вы часто выполняете запросы, основанные на названиях район